# Summarizing Retail Data - Grouping and aggregating data

For this notebook, we will be exploring the techniques for grouping and aggregating data. We will use data from https://archive.ics.uci.edu/ml/datasets/online+retail

Table of contents:
- Speeding up data loading with Feather
- Grouping data using pandas library
- Plotting Aggregations
- Limiting Groups
- Aggregating data using built-in functions
- Manipulating Plot
